In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from time import sleep


from scraper import ScanScraper

scraper = ScanScraper()
scraper._navigate_to_3060_cards()

'https://www.scan.co.uk/shop/computer-hardware/gpu-nvidia-gaming/nvidia-geforce-rtx-3060-graphics-cards'

In [3]:
container = scraper.driver.find_element(By.XPATH, "//ul[@class='productColumns']")


In [4]:
list_of_3060_cards = container.find_elements(By.XPATH, './li')


In [ ]:
# len(list_of_3060_cards)

In [5]:
hidden_products = scraper.driver.find_elements(By.XPATH, "//li[contains(@data-price, '999999.00')]")


In [ ]:
# len(hidden_products)


In [7]:
def non_match_elements(list_a, list_b):
    non_match = []
    for i in list_a:
        if i not in list_b:
            non_match.append(i)
    return non_match


in_stock_3060_cards = non_match_elements(list_of_3060_cards, hidden_products)


In [26]:
# len(in_stock_3060_cards)


20

In [8]:
list_of_links_for_3060 = []
for rtx_3060 in in_stock_3060_cards:
    list_of_links_for_3060.append(rtx_3060.find_element(By.TAG_NAME, 'a').get_attribute('href'))

In [9]:
from selenium.common.exceptions import NoSuchElementException
import json
import pandas as pd
import requests
import os
import shutil
import uuid
import urllib

root_dir = os.getcwd()
raw_data = os.path.join(root_dir, 'raw_data')
is_raw_data_there = os.path.exists(raw_data)

if is_raw_data_there == True:
    shutil.rmtree(raw_data)
else:
    os.makedirs(raw_data)

product_list = []
Product_Image_URL = []

for link in list_of_links_for_3060[0:2]:

    product_dictionary = {
        'Product Name': [],
        'SKU': [],
        'Unique ID': [],
        'Price': [],
        'Link': [],
        'Product Image URL': []
    }

    scraper.driver.get(link)
    sleep(1)
    product_dictionary['Link'].append(link)

    # Get Product Name
    try:
        product_name = scraper.driver.find_element(By.XPATH, "//h1[@itemprop='name']")
        product_dictionary['Product Name'].append(product_name.text)
    except NoSuchElementException:
        product_dictionary['Product Name'].append('N/A')

    # Get Product SKU/Friendly ID
    try:
        sku = scraper.driver.find_element(By.XPATH, "(//strong[@itemprop='sku'])[1]")
        product_dictionary['SKU'].append(sku.text)
    except NoSuchElementException:
        product_dictionary['SKU'].append('N/A')

    # Generate UUID (Unique)
    try:
        unique_id = uuid.uuid4()
        str(unique_id)
        product_dictionary['Unique ID'].append(str(unique_id))
    except:
        product_dictionary['Unique ID'].append('N/A')

    # Get Image URL
    try:
        element = scraper.driver.find_element(By.XPATH, "//img[@class='zoomable-image'][1]")
        image_url = element.get_attribute('src')
        Product_Image_URL.append(image_url)
        product_dictionary['Product Image URL'].append(image_url)
    except NoSuchElementException:
        product_dictionary['Product Image URL'].append('N/A')

    # Item Price
    try:
        price = scraper.driver.find_element(By.XPATH, "(//span[@class='price'])[4]")
        product_dictionary['Price'].append(price.text)
    except NoSuchElementException:
        product_dictionary['Price'].append('N/A')

    # append each product dictionary to a list.
    product_list.append(product_dictionary)

    # Create a folder for each product, named after its SKU.
    product_entries = os.path.join(root_dir, raw_data, f"{(sku.text)}")
    os.makedirs(product_entries)
    with open(f'{product_entries}\data.json', 'w') as fp:
        json.dump(product_dictionary, fp)
    
    # Create an image Folder for each image
    images_folder = os.path.join(product_entries, "Images")  
    os.makedirs(images_folder)
    
    # Overcome 'HTTPError: HTTP Error 403: Forbidden' error code by modifying 'user-agent' variable that is send with the request.
    opener=urllib.request.build_opener()
    opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.115 Safari/537.36')]
    urllib.request.install_opener(opener)
    urllib.request.urlretrieve(image_url, f'{images_folder}\{sku.text}.jpg')


In [ ]:
# product_list

In [10]:
pd.DataFrame(product_list)


,Product Name,SKU,Unique ID,Price,Link,Product Image URL
0,[ZOTAC NVIDIA GeForce RTX 3060 12GB TWIN EDGE ...,[LN113664],[922750b1-a75f-400c-97d9-413bc743af9f],[£394.99],[https://www.scan.co.uk/products/zotac-nvidia-...,[https://www.scan.co.uk/images/products/327389...
1,[EVGA NVIDIA GeForce RTX 3060 12GB XC GAMING A...,[LN114676],[5cb911a9-e213-4871-9382-da1c5024eb62],[£429.98],[https://www.scan.co.uk/products/evga-nvidia-g...,[https://www.scan.co.uk/images/products/328906...


In [13]:
# import requests
# with open('lol.jpg', 'wb') as f:
#     f.write(requests.get("https://apod.nasa.gov/apod/image/1701/potw1636aN159_HST_2048.jpg").content)
#     f.close()


In [1]:
product_dictionary = {
        'Product Name': [],
        'SKU': [],
        'Unique ID': [],
        'Price': [],
        'Link': [],
        'Product Image URL': []
    }

In [2]:
len(product_dictionary)

6